# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 8 2022 8:30AM,243605,10,0085925304,ISDIN Corporation,Released
1,Aug 8 2022 8:30AM,243605,10,0085925374,ISDIN Corporation,Released
2,Aug 8 2022 8:30AM,243605,10,0085925433,ISDIN Corporation,Released
3,Aug 8 2022 8:30AM,243605,10,0085925542,ISDIN Corporation,Released
4,Aug 8 2022 8:30AM,243605,10,0085925543,ISDIN Corporation,Released
5,Aug 8 2022 8:30AM,243605,10,0085925557,ISDIN Corporation,Released
6,Aug 8 2022 8:30AM,243605,10,0085925581,ISDIN Corporation,Released
7,Aug 8 2022 8:30AM,243605,10,0085925583,ISDIN Corporation,Released
8,Aug 8 2022 8:30AM,243605,10,0085925596,ISDIN Corporation,Released
9,Aug 8 2022 8:30AM,243605,10,0085925594,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,243602,Released,1
13,243603,Released,2
14,243604,Released,1
15,243605,Released,12
16,243606,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243602,NaN,1.0
243603,NaN,2.0
243604,NaN,1.0
243605,NaN,12.0
243606,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243584,0.0,1.0
243587,0.0,5.0
243588,0.0,1.0
243589,0.0,1.0
243590,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243584,0,1
243587,0,5
243588,0,1
243589,0,1
243590,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243584,0,1
1,243587,0,5
2,243588,0,1
3,243589,0,1
4,243590,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243584,,1
1,243587,,5
2,243588,,1
3,243589,,1
4,243590,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 8 2022 8:30AM,243605,10,ISDIN Corporation
12,Aug 8 2022 8:30AM,243601,10,ISDIN Corporation
17,Aug 5 2022 6:20PM,243606,21,"NBTY Global, Inc."
18,Aug 5 2022 3:51PM,243603,20,Alumier Labs Inc.
20,Aug 5 2022 3:50PM,243602,20,Alumier Labs Inc.
21,Aug 5 2022 3:48PM,243599,20,Alumier Labs Inc.
31,Aug 5 2022 3:47PM,243598,20,Alumier Labs Inc.
35,Aug 5 2022 3:47PM,243604,19,"GUSA Granules USA, Inc."
36,Aug 5 2022 3:46PM,243597,20,Alumier Labs Inc.
45,Aug 5 2022 3:40PM,243600,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 8 2022 8:30AM,243605,10,ISDIN Corporation,,12
1,Aug 8 2022 8:30AM,243601,10,ISDIN Corporation,,5
2,Aug 5 2022 6:20PM,243606,21,"NBTY Global, Inc.",1,
3,Aug 5 2022 3:51PM,243603,20,Alumier Labs Inc.,,2
4,Aug 5 2022 3:50PM,243602,20,Alumier Labs Inc.,,1
5,Aug 5 2022 3:48PM,243599,20,Alumier Labs Inc.,,10
6,Aug 5 2022 3:47PM,243598,20,Alumier Labs Inc.,,4
7,Aug 5 2022 3:47PM,243604,19,"GUSA Granules USA, Inc.",,1
8,Aug 5 2022 3:46PM,243597,20,Alumier Labs Inc.,,9
9,Aug 5 2022 3:40PM,243600,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 8:30AM,243605,10,ISDIN Corporation,12,
1,Aug 8 2022 8:30AM,243601,10,ISDIN Corporation,5,
2,Aug 5 2022 6:20PM,243606,21,"NBTY Global, Inc.",,1
3,Aug 5 2022 3:51PM,243603,20,Alumier Labs Inc.,2,
4,Aug 5 2022 3:50PM,243602,20,Alumier Labs Inc.,1,
5,Aug 5 2022 3:48PM,243599,20,Alumier Labs Inc.,10,
6,Aug 5 2022 3:47PM,243598,20,Alumier Labs Inc.,4,
7,Aug 5 2022 3:47PM,243604,19,"GUSA Granules USA, Inc.",1,
8,Aug 5 2022 3:46PM,243597,20,Alumier Labs Inc.,9,
9,Aug 5 2022 3:40PM,243600,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 8:30AM,243605,10,ISDIN Corporation,12,
1,Aug 8 2022 8:30AM,243601,10,ISDIN Corporation,5,
2,Aug 5 2022 6:20PM,243606,21,"NBTY Global, Inc.",,1
3,Aug 5 2022 3:51PM,243603,20,Alumier Labs Inc.,2,
4,Aug 5 2022 3:50PM,243602,20,Alumier Labs Inc.,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 8:30AM,243605,10,ISDIN Corporation,12.0,NaN
1,Aug 8 2022 8:30AM,243601,10,ISDIN Corporation,5.0,NaN
2,Aug 5 2022 6:20PM,243606,21,"NBTY Global, Inc.",NaN,1.0
3,Aug 5 2022 3:51PM,243603,20,Alumier Labs Inc.,2.0,NaN
4,Aug 5 2022 3:50PM,243602,20,Alumier Labs Inc.,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 8:30AM,243605,10,ISDIN Corporation,12.0,0.0
1,Aug 8 2022 8:30AM,243601,10,ISDIN Corporation,5.0,0.0
2,Aug 5 2022 6:20PM,243606,21,"NBTY Global, Inc.",0.0,1.0
3,Aug 5 2022 3:51PM,243603,20,Alumier Labs Inc.,2.0,0.0
4,Aug 5 2022 3:50PM,243602,20,Alumier Labs Inc.,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,974387,24.0,0.0
16,487171,6.0,0.0
19,730781,3.0,0.0
20,1217999,26.0,0.0
21,243606,0.0,1.0
22,487196,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,974387,24.0,0.0
1,16,487171,6.0,0.0
2,19,730781,3.0,0.0
3,20,1217999,26.0,0.0
4,21,243606,0.0,1.0
5,22,487196,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,24.0,0.0
1,16,6.0,0.0
2,19,3.0,0.0
3,20,26.0,0.0
4,21,0.0,1.0
5,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,16,Released,6.0
2,19,Released,3.0
3,20,Released,26.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Executing,0.0,0.0,0.0,0.0,1.0,0.0
Released,24.0,6.0,3.0,26.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Executing,0.0,0.0,0.0,0.0,1.0,0.0
1,Released,24.0,6.0,3.0,26.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Executing,0.0,0.0,0.0,0.0,1.0,0.0
1,Released,24.0,6.0,3.0,26.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()